In [ ]:
import requests
import numpy as np
import datetime

Set up your API token from your profile on observation portal

In [ ]:
API_TOKEN = 'XXX'
PROPOSAL_ID = 'LCOEPO2019A-001'

In [ ]:
# Determine when the observations are to take place
# The period is NOT the period of the variable. 
# The period is the cadence at which you want to observe, in HOURS
# The Jitter is the window within which your observation may take place. 

target_known_period = 'XXX'
observation_period = 'XXX'

# Want jitter to equal period to "fully cover" time
jitter = observation_period

cadence = {
    'start': '2019-10-05 18:00:00',
    'end': '2019-10-06 05:00:00',
    'period': observation_period,
    'jitter': jitter
}

In [ ]:
target = {
    'name': 'Target_XXX',
    'type': 'ICRS',
    'ra': 'XXX',
    'dec': 'XXX',
    'epoch': 2000
}

In [ ]:
# Keep Max_airmass at 1.6
# min_lunar_distance is in DEGREES, keep at 20 degrees

constraints = {
    'max_airmass': 2.0,
    'min_lunar_distance': 20
}

In [ ]:
# This does not need to be changed.
# Exposure time is in SECONDS

configurations = [
    {
        'type': 'EXPOSE',
        'instrument_type': '0M4-SCICAM-SBIG',
        'target': target,
        'constraints': constraints,
        'acquisition_config': {},
        'guiding_config': {},
        'instrument_configs': [
            {
                'exposure_time': 90,
                'exposure_count': 1,
                'optical_elements': {
                    'filter': 'v'
                }
            }
        ]
    },
    {
        'type': 'EXPOSE',
        'instrument_type': '0M4-SCICAM-SBIG',
        'target': target,
        'constraints': constraints,
        'acquisition_config': {},
        'guiding_config': {},
        'instrument_configs': [
            {
                'exposure_time': 90,
                'exposure_count': 1,
                'optical_elements': {
                    'filter': 'b'
                }
            }
        ]
    }
]

In [ ]:
# This cell does not need to be edited.

# We do not provide windows for a cadence request
windows = []

# The telescope class that should be used for this observation
location = {
    'telescope_class': '0m4'
}

In [ ]:
# Please input the name of your observation request. Eg. Group_Star_Target_B
requestgroup = {
    'name': 'XXX',  # The title (group name)
    'proposal': PROPOSAL_ID,
    'ipp_value': 1.05,
    'operator': 'SINGLE',
    'observation_type': 'NORMAL',
    'requests': [{
        'cadence': cadence,
        'configurations': configurations,
        'windows': windows,
        'location': location,
    }]
}

# Below cells allows you to submit your request to LCO to be checked for errors. This does not actually submit the observing to the telescopes. In order to run the below cells, you will need access to the internet to send your requst to LCO to be checked.

In [ ]:
response = requests.post(
    'https://observe.lco.global/api/requestgroups/cadence/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=requestgroup  # Make sure you use json!
)

In [ ]:
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('Request failed: {}'.format(response.content))
    raise exc

In [ ]:
cadence_request = response.json()

print('Cadence generated {} requests'.format(len(cadence_request['requests'])))
i = 1
for request in cadence_request['requests']:
    print('Request {0} window start: {1} window end: {2}'.format(
        i, request['windows'][0]['start'], request['windows'][0]['end']
    ))
    i = i + 1

In [ ]:
# Submit the fully formed RequestGroup
response = requests.post(
    'https://observe.lco.global/api/requestgroups/validate/',
    headers={'Authorization': 'Token {}'.format(API_TOKEN)},
    json=cadence_request  # Make sure you use json!
)

# Make sure the API call was successful
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as exc:
    print('API call failed: {}'.format(response.content))
    raise exc

requestgroup_dict = response.json()  # The API will return the newly submitted requestgroup as json

print('Total number of observing hours needed for this request: ',
      requestgroup_dict['request_durations']['duration']/60./60)

# Once you have everything looking fine, please find an instructor to check it and they will submit the request together with you if internet is available, or for you later if the internet is not currently available. 